# Imports

In [ ]:
import pandas as pd
import numpy as np
# Suppress the SettingWithCopyWarning 
pd.options.mode.chained_assignment = None
import os

pd.set_option('display.max_columns', None)
df_inflation = pd.read_csv('df_inflation.csv')
df_inflation.head()

In [ ]:

# Read in the Excel file
df_wide_data = pd.read_excel("DataPreparationOutputWeird.xlsx")
df_wide_data = df_wide_data[df_wide_data['GLPSalesUoM'] != 0]
# Display the first few rows of the dataframe
df_wide_data.head()

In [ ]:
df_wide_data.info()

# Data checks

Below we check for abnormally high prices. This is an error we had before. 

In [ ]:
count = (df_wide_data['PriceSalesUoMEUR'] > 100000).sum()
print("Number of rows where PriceSalesUoMEUR > 100000:", count)

In [ ]:
len(df_wide_data)

In [ ]:
df_wide_data['ItemNumber'].nunique()

In [ ]:
df_wide_data['CustomerSoldTo'].nunique()

In [ ]:
df_wide_data = df_wide_data.drop(columns=['OrderDate_y']).rename(columns={'OrderDate_x': 'OrderDate'})

In [ ]:
oldest_date = df_wide_data['OrderDate'].min()
latest_date = df_wide_data['OrderDate'].max()

print("Oldest Date:", oldest_date)
print("Latest Date:", latest_date)

# Add features

In [ ]:
df_wide_data['Discount'] = df_wide_data['GLPSalesUoM'] - df_wide_data['PriceSalesUoMEUR']
df_wide_data['Discount'] = df_wide_data['Discount'].astype(float)
df_wide_data.loc[df_wide_data['GLPSalesUoM'] == 0, 'Discount'] = 0
df_wide_data.loc[df_wide_data['PriceSalesUoMEUR'] == 0, 'Discount'] = 0

In [ ]:
df_wide_data['ManualDiscount'] = np.where(
    df_wide_data['PriceSalesUoMEUR'] == 0,
    0,
    df_wide_data['PCoverPriceSalesUoM'] - df_wide_data['PriceSalesUoMEUR']
)

In [ ]:
df_wide_data['DaysDifference'] = (df_wide_data['ActualShipDate'] - df_wide_data['OriginalPromisedShipDate']).dt.days

In [ ]:
df_wide_data['LatestDaysDifference'] = (df_wide_data['ActualShipDate'] - df_wide_data['LatestPromisedShipDate']).dt.days

# Take only items with variable GLP

In [ ]:
def check_unique_glp(df):
    items_w_more_than_1_glp = 0
    items_w_1_glp =0
    items_with_multiple_glp = []
    # Group by ItemNumber and find the number of unique PriceSalesUoMEUR values for each
    unique__glp_per_item = df.groupby('ItemNumber')['GLPSalesUoM'].nunique()

    # Print the result
    print("Number of unique GLPSalesUoM values per ItemNumber:")
    for item, unique_price in unique__glp_per_item.items():
        if(unique_price)>1:
            items_w_more_than_1_glp += 1
            items_with_multiple_glp.append(item)
        else:
            items_w_1_glp += 1
        #print(f"ItemNumber: {item}, Unique GLPSalesUoM: {unique_price}")
    print(f"There are {items_w_more_than_1_glp} items with more than 1 glp")
    print(f"There are {items_w_1_glp} items with exactly 1 glp")

    return items_with_multiple_glp

In [ ]:
unique_item_numbers = df_wide_data['ItemNumber'].nunique()
print(f'There are {unique_item_numbers} unique ItemNumbers in df_wide_data.')

In [ ]:
var_glp_items = check_unique_glp(df_wide_data)

In [ ]:
num_rows = df_wide_data[df_wide_data['ItemNumber'].isin(var_glp_items)].shape[0]
print("Number of rows with ItemNumber in var_glp_items:", num_rows)

In [ ]:
df_wide_data_var_glp = df_wide_data[df_wide_data['ItemNumber'].isin(var_glp_items)]
df_wide_data_var_glp_sorted = df_wide_data_var_glp.sort_values(by=['ItemNumber', 'CustomerSoldTo','OrderDate'])

# Split df by region

From this point onwards, we split data into multiple dataframes using region. This is why every operation will be done a number of times. We generate one dataframe for USA, one for APA, one for EU, and one for EU and APA together.

In [ ]:
df_wide_data_usa = df_wide_data[df_wide_data['GeographicRegion'] == 'USA']
df_wide_data_usa.head()

In [ ]:
var_glp_items_usa = check_unique_glp(df_wide_data_usa)

In [ ]:
df_wide_data_var_glp_usa = df_wide_data_usa[df_wide_data_usa['ItemNumber'].isin(var_glp_items_usa)]
df_wide_data_var_glp_sorted_usa = df_wide_data_var_glp_usa.sort_values(by=['ItemNumber', 'CustomerSoldTo','OrderDate'])

In [ ]:
df_wide_data_eu_apa = df_wide_data[df_wide_data['GeographicRegion'] != 'USA']
df_wide_data_eu_apa.head()

In [ ]:
df_wide_data_eu = df_wide_data[df_wide_data['GeographicRegion'] == 'EUR']
df_wide_data_apa = df_wide_data[df_wide_data['GeographicRegion'] == 'APA']

In [ ]:
var_glp_items_eu_apa = check_unique_glp(df_wide_data_eu_apa)

In [ ]:
var_glp_items_eu = check_unique_glp(df_wide_data_eu)
var_glp_items_apa = check_unique_glp(df_wide_data_apa)

In [ ]:
df_wide_data_var_glp_eu_apa = df_wide_data_eu_apa[df_wide_data_eu_apa['ItemNumber'].isin(var_glp_items_eu_apa)]
df_wide_data_var_glp_sorted_eu_apa = df_wide_data_var_glp_eu_apa.sort_values(by=['ItemNumber', 'CustomerSoldTo','OrderDate'])

In [ ]:
df_wide_data_var_glp_eu = df_wide_data_eu[df_wide_data_eu['ItemNumber'].isin(var_glp_items_eu)]
df_wide_data_var_glp_sorted_eu = df_wide_data_var_glp_eu.sort_values(by=['ItemNumber', 'CustomerSoldTo','OrderDate'])

In [ ]:
df_wide_data_var_glp_apa = df_wide_data_apa[df_wide_data_apa['ItemNumber'].isin(var_glp_items_apa)]
df_wide_data_var_glp_sorted_apa = df_wide_data_var_glp_apa.sort_values(by=['ItemNumber', 'CustomerSoldTo','OrderDate'])

In [ ]:
df_wide_data_var_glp_sorted_eu

# save region split order data

By this point, the data is not yet aggregated per product. It is still long and wide order data. We save the region splits in case future changes are necessary in one of the splits.

In [ ]:
# Save the DataFrame to a pickle file
df_wide_data_var_glp_sorted_apa.to_pickle('PreparedOrderDataAPA.pkl')
df_wide_data_var_glp_sorted_eu_apa.to_pickle('PreparedOrderDataEUandAPA.pkl')
df_wide_data_var_glp_sorted_eu.to_pickle('PreparedOrderDataEU.pkl')
df_wide_data_var_glp_sorted_usa.to_pickle('PreparedOrderDataUSA.pkl')

# add inflation

In [ ]:
# Merge df_sales_variable_glp_sorted with df_inflation on OrderYear and OrderMonth
df_wide_data_var_glp_sorted_inflation = df_wide_data_var_glp_sorted.merge(
    df_inflation[['Year', 'Month', 'InflationIndex']],
    left_on=['OrderYear', 'OrderMonth'],
    right_on=['Year', 'Month'],
    how='left'
)

# Drop the redundant 'Year' and 'Month' columns from the merged dataframe
df_wide_data_var_glp_sorted_inflation.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
# Merge df_sales_variable_glp_sorted with df_inflation on OrderYear and OrderMonth
df_wide_data_var_glp_sorted_inflation_eu_apa = df_wide_data_var_glp_sorted_eu_apa.merge(
    df_inflation[['Year', 'Month', 'InflationIndex']],
    left_on=['OrderYear', 'OrderMonth'],
    right_on=['Year', 'Month'],
    how='left'
)

# Drop the redundant 'Year' and 'Month' columns from the merged dataframe
df_wide_data_var_glp_sorted_inflation_eu_apa.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
# Merge df_sales_variable_glp_sorted with df_inflation on OrderYear and OrderMonth
df_wide_data_var_glp_sorted_inflation_usa = df_wide_data_var_glp_sorted_usa.merge(
    df_inflation[['Year', 'Month', 'InflationIndex']],
    left_on=['OrderYear', 'OrderMonth'],
    right_on=['Year', 'Month'],
    how='left'
)

# Drop the redundant 'Year' and 'Month' columns from the merged dataframe
df_wide_data_var_glp_sorted_inflation_usa.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
# Merge df_sales_variable_glp_sorted with df_inflation on OrderYear and OrderMonth
df_wide_data_var_glp_sorted_inflation_eu = df_wide_data_var_glp_sorted_eu.merge(
    df_inflation[['Year', 'Month', 'InflationIndex']],
    left_on=['OrderYear', 'OrderMonth'],
    right_on=['Year', 'Month'],
    how='left'
)

# Drop the redundant 'Year' and 'Month' columns from the merged dataframe
df_wide_data_var_glp_sorted_inflation_eu.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
# Merge df_sales_variable_glp_sorted with df_inflation on OrderYear and OrderMonth
df_wide_data_var_glp_sorted_inflation_apa = df_wide_data_var_glp_sorted_apa.merge(
    df_inflation[['Year', 'Month', 'InflationIndex']],
    left_on=['OrderYear', 'OrderMonth'],
    right_on=['Year', 'Month'],
    how='left'
)

# Drop the redundant 'Year' and 'Month' columns from the merged dataframe
df_wide_data_var_glp_sorted_inflation_apa.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
df_wide_data_var_glp_sorted_inflation_usa.info()

# Aggregate per item-glp
The following function is the first step in computing the price elasticity values. For each item, for each GLP that item had, we compute the time that GLP was active, the average inflation index, and the quantity sold at that GLP. We also compute some features, but the numerical ones will be further averaged to product level. 

In [ ]:
def aggregate_per_item_glp(group_item_glp):
    group_item_glp = group_item_glp.sort_values(by='OrderDate')
    aggredated_data = [] 
    start_time = pd.to_datetime(group_item_glp['OrderDate'].iloc[0])
    start_year = start_time.year
    end_time = pd.to_datetime(group_item_glp['OrderDate'].iloc[-1])
    end_year = end_time.year
    #effective_from = pd.to_datetime(group_item_glp['EffectiveFrom'].iloc[0])
    #effective_to = pd.to_datetime(group_item_glp['EffectiveTo'].iloc[-1])
    oem = group_item_glp['OEMName'].iloc[0]
    #time_effective = (effective_to - effective_from).days
    #if effective_to > pd.Timestamp('2025-04-01'):
        #today = pd.Timestamp('2025-04-01')
        #time_effective = (today - effective_from).days
    time_active = (end_time - start_time).days
    avg_inflation_index = group_item_glp['InflationIndex'].mean()
    total_quantity = group_item_glp['OrderQuantity'].sum()
    group_glp = group_item_glp['GLPSalesUoM'].iloc[0]
    group_item = group_item_glp['ItemNumber'].iloc[0]
    avg_sales_price = group_item_glp['PriceSalesUoMEUR'][group_item_glp['PriceSalesUoMEUR'] != 0].mean()
    avg_cost_price = group_item_glp['CostSalesUoMEUR'].mean()
    manual_discount = group_item_glp['ManualDiscount'].mean()
    # consolidation_customer = group_item_glp['ConsolidationCustomer'].mean()
    supplier_leadtime = group_item_glp['SupplierLeadtime'].mean()
    price_category = group_item_glp['PriceCategory'].iloc[0]
    percent_eu = (group_item_glp['GeographicRegion'] == 'EUR').mean() * 100
    percent_usa = (group_item_glp['GeographicRegion'] == 'USA').mean() * 100
    percent_apa = (group_item_glp['GeographicRegion'] == 'APA').mean() * 100
    mean_days_diff = group_item_glp['DaysDifference'].mean()
    mean_latest_days_diff = group_item_glp['LatestDaysDifference'].mean()
    VIEngineered = group_item_glp['VIEngineered'].iloc[0]
    SparePartsCategory = group_item_glp['SparePartsCategory'].iloc[0]
    TechnicalClassification = group_item_glp['TechnicalClassification'].iloc[0]
    split_columns = group_item_glp['TechnicalClassification'].str.split('-', expand=True)
    split_columns = split_columns.reindex(columns=range(5))
    split_columns.columns = ['level1', 'level2', 'level3', 'level4', 'level5']
    

    if time_active == 0:
        time_active = 1
    #if time_effective == 0:
        #time_effective = 1 
        
    aggredated_data.append({ 
        'ItemNumber': group_item,
        'GLPSalesUoM': group_glp,
        'StartTime': start_time,
        'EndTime': end_time,
        'StartYear': start_year,
        'EndYear': end_year,
        'TimeActive': time_active,
        #'EffectiveFrom': effective_from,
        #'EffectiveTo': effective_to,
        #'TimeEffective': time_effective,
        'TotalQuantity': total_quantity,
        'AvgInflationIndex': avg_inflation_index,
        'AvgSalesPrice': avg_sales_price,
        'AvgCostPrice': avg_cost_price,
        'ManualDiscount': manual_discount,
        # 'ConsolidationCustomer': consolidation_customer,
        'SupplierLeadtime': supplier_leadtime,
        'PriceCategory': price_category,
        'PercentEU': percent_eu,
        'PercentUSA': percent_usa,
        'PercentAPA': percent_apa,
        'OEM': oem,
        'MeanDaysDifference': mean_days_diff,
        'MeanLatestDaysDifference': mean_latest_days_diff,
        'VIEngineered': VIEngineered,
        'SparePartsCategory': SparePartsCategory,
        'TechnicalClassification': TechnicalClassification,
        'Level1': split_columns['level1'].iloc[0],
        'Level2': split_columns['level2'].iloc[0],
        'Level3': split_columns['level3'].iloc[0],
        'Level4': split_columns['level4'].iloc[0],
        'Level5': split_columns['level5'].iloc[0]


    })

    return pd.DataFrame(aggredated_data)

In [ ]:
aggreagated_item_glp = df_wide_data_var_glp_sorted_inflation.sort_values(by=['OrderDate']).groupby(['ItemNumber', 'GLPSalesUoM']).apply(aggregate_per_item_glp)
ungrouped_aggregated_item_glp= aggreagated_item_glp.reset_index(drop=True)
item_grouped_aggregated = ungrouped_aggregated_item_glp.groupby('ItemNumber')
item_grouped_aggregated_sorted = item_grouped_aggregated.apply(lambda x: x.sort_values(by='StartTime')).reset_index(drop=True)


In [ ]:
item_grouped_aggregated_sorted.head(20)

In [ ]:
aggreagated_item_glp_usa = df_wide_data_var_glp_sorted_inflation_usa.groupby(['ItemNumber', 'GLPSalesUoM']).apply(aggregate_per_item_glp)
ungrouped_aggregated_item_glp_usa= aggreagated_item_glp_usa.reset_index(drop=True)
item_grouped_aggregated_usa = ungrouped_aggregated_item_glp_usa.groupby('ItemNumber')
item_grouped_aggregated_sorted_usa = item_grouped_aggregated_usa.apply(lambda x: x.sort_values(by='StartTime')).reset_index(drop=True)

In [ ]:
aggreagated_item_glp_eu_apa = df_wide_data_var_glp_sorted_inflation_eu_apa.sort_values(by=['OrderDate']).groupby(['ItemNumber', 'GLPSalesUoM']).apply(aggregate_per_item_glp)
ungrouped_aggregated_item_glp_eu_apa= aggreagated_item_glp_eu_apa.reset_index(drop=True)
item_grouped_aggregated_eu_apa = ungrouped_aggregated_item_glp_eu_apa.groupby('ItemNumber')
item_grouped_aggregated_sorted_eu_apa = item_grouped_aggregated_eu_apa.apply(lambda x: x.sort_values(by='StartTime')).reset_index(drop=True)

In [ ]:
aggreagated_item_glp_eu = df_wide_data_var_glp_sorted_inflation_eu.sort_values(by=['OrderDate']).groupby(['ItemNumber', 'GLPSalesUoM']).apply(aggregate_per_item_glp)
ungrouped_aggregated_item_glp_eu= aggreagated_item_glp_eu.reset_index(drop=True)
item_grouped_aggregated_eu = ungrouped_aggregated_item_glp_eu.groupby('ItemNumber')
item_grouped_aggregated_sorted_eu = item_grouped_aggregated_eu.apply(lambda x: x.sort_values(by='StartTime')).reset_index(drop=True)

In [ ]:
aggreagated_item_glp_apa = df_wide_data_var_glp_sorted_inflation_apa.sort_values(by=['OrderDate']).groupby(['ItemNumber', 'GLPSalesUoM']).apply(aggregate_per_item_glp)
ungrouped_aggregated_item_glp_apa= aggreagated_item_glp_apa.reset_index(drop=True)
item_grouped_aggregated_apa = ungrouped_aggregated_item_glp_apa.groupby('ItemNumber')
item_grouped_aggregated_sorted_apa = item_grouped_aggregated_apa.apply(lambda x: x.sort_values(by='StartTime')).reset_index(drop=True)

In [ ]:
item_grouped_aggregated_eu.head(50)

# Calculate price elasticity
The following function computes the array of price elasticities per product, at every point where the GLP changes. This results in a dataframe with a lot of redundancy, with one row per product-elasticity. It will be condensed further into one row per product.

In [ ]:
def calculate_pe(group_item_aggr):
    pe_data = []
    group_item = group_item_aggr['ItemNumber'].iloc[0]
    avg_sales_price = group_item_aggr['AvgSalesPrice'].mean()
    avg_cost_price = group_item_aggr['AvgCostPrice'].mean()
    avg_manual_discount = group_item_aggr['ManualDiscount'].mean()
    #avg_consolidation_customer = group_item_aggr['ConsolidationCustomer'].mean()
    avg_supplier_leadtime = group_item_aggr['SupplierLeadtime'].mean()
    price_category = group_item_aggr['PriceCategory'].iloc[0]
    percent_eu = group_item_aggr['PercentEU'].mean()
    percent_usa = group_item_aggr['PercentUSA'].mean()
    percent_apa = group_item_aggr['PercentAPA'].mean()
    VIEngineered = group_item_aggr['VIEngineered'].iloc[0]
    SparePartsCategory = group_item_aggr['SparePartsCategory'].iloc[0]
    TechnicalClassification = group_item_aggr['TechnicalClassification'].iloc[0]
    OEM = group_item_aggr['OEM'].iloc[0]
    level1 = group_item_aggr['Level1'].iloc[0]
    level2 = group_item_aggr['Level2'].iloc[0]
    level3 = group_item_aggr['Level3'].iloc[0]
    level4 = group_item_aggr['Level4'].iloc[0]
    level5 = group_item_aggr['Level5'].iloc[0]

    #avgdaysdifferece = group_item_aggr['MeanDaysDifference'].mean()
    #avglatestdaysdifferece = group_item_aggr['MeanLatestDaysDifference'].mean()

    previous_row = None

    for _, row in group_item_aggr.iterrows():
        qty_current_price = row['TotalQuantity']
        current_glp = row['GLPSalesUoM']
        current_start_time = row['StartTime']
        current_end_time = row['EndTime']
        current_start_year = row['StartYear']
        current_end_year = row['EndYear']
        current_inflation = row['AvgInflationIndex']
        current_time_active = row['TimeActive']
        current_qty_per_day = qty_current_price / current_time_active

        
        # if we are at the first row (first ever glp) in the item group
        if previous_row is None:
            qty_previous_price = None
            previous_glp = None
            previous_inflation = None
            previous_time_active = None
            time_adj_pe = None
            infl_time_adj_pe = None
            qty_change = None
            glp_change = None
            perc_time_adj_qty_change = None
            perc_glp_change = None
            previous_qty_per_day = None
            time_adj_qty_change = None
            inflation_ratio = None
            adjusted_current_glp = None
            perc_inflation_adjusted_glp_change = None
        else:
            qty_previous_price = previous_row['TotalQuantity']
            previous_glp = previous_row['GLPSalesUoM']
            previous_inflation = previous_row['AvgInflationIndex']
            previous_time_active = previous_row['TimeActive']
            previous_qty_per_day = qty_previous_price / previous_time_active

            qty_change = qty_current_price - qty_previous_price
            time_adj_qty_change = current_qty_per_day - previous_qty_per_day
            perc_time_adj_qty_change = time_adj_qty_change / previous_qty_per_day

            glp_change = current_glp - previous_glp
            perc_glp_change = glp_change / previous_glp

            time_adj_pe = perc_time_adj_qty_change / perc_glp_change

            if previous_inflation == 0:
                previous_inflation = 0.01
            
            if current_inflation == 0:
                current_inflation = 0.01
                
            inflation_ratio = current_inflation / previous_inflation
            adjusted_current_glp = current_glp * inflation_ratio
            perc_inflation_adjusted_glp_change = (adjusted_current_glp - previous_glp) / previous_glp
            if perc_inflation_adjusted_glp_change == 0:
                perc_inflation_adjusted_glp_change = 0.0001
            infl_time_adj_pe = perc_time_adj_qty_change / perc_inflation_adjusted_glp_change

        pe_data.append({
            'ItemNumber': group_item,
            'MeanSalesPrice': avg_sales_price,
            'MeanCostPrice': avg_cost_price,
            'MeanManualDiscount': avg_manual_discount,
            #'AvgConsolidationCustomer': avg_consolidation_customer,
            'MeanSupplierLeadtime': avg_supplier_leadtime,
            'PriceCategory': price_category,
            'CurrentGLP': current_glp,
            'PreviousGLP': previous_glp,
            'CurrentStartTime': current_start_time,
            'CurrentEndTime': current_end_time,
            'CurrentStartYear': current_start_year,
            'CurrentEndYear': current_end_year,
            'CurrentInflation': current_inflation,
            'PreviousInflation': previous_inflation,
            'CurrentTimeActive': current_time_active,
            'PreviousTimeActive': previous_time_active,
            'CurrentQuantity': qty_current_price,
            'PreviousQuantity': qty_previous_price,
            'CurrentDailyQty': current_qty_per_day,
            'PreviousDailyQty': previous_qty_per_day,
            'QuantityChange': qty_change,
            'DailyQtyChange': time_adj_qty_change,
            'PercDailyQtyChange': perc_time_adj_qty_change,
            'GLPChange': glp_change,
            'PercGLPChange': perc_glp_change,
            'TimeAdjPriceElasticity': time_adj_pe,
            'InflationRatio': inflation_ratio,
            'DeflatedCurrentGLP': adjusted_current_glp,
            'PercInflationAdjustedGLPChange': perc_inflation_adjusted_glp_change,
            'InflationTimeAdjPriceElasticity': infl_time_adj_pe,
            'PercentEU': percent_eu,
            'PercentUSA': percent_usa,
            'PercentAPA': percent_apa,
            'VIEngineered': VIEngineered,
            'SparePartsCategory': SparePartsCategory,
            'TechnicalClassification': TechnicalClassification,
            'OEM': OEM,
            'Level1': level1,
            'Level2': level2,
            'Level3': level3,
            'Level4': level4,
            'Level5': level5,
            #'MeanDaysDifference': avgdaysdifferece,
            #'MeanLatestDaysDifference': avglatestdaysdifferece
        })

        previous_row = row
    
    return pd.DataFrame(pe_data)

In [ ]:
calculated_pe = item_grouped_aggregated_sorted.groupby("ItemNumber").apply(calculate_pe)

In [ ]:
calculated_pe_usa = item_grouped_aggregated_sorted_usa.groupby("ItemNumber").apply(calculate_pe)

In [ ]:
calculated_pe_eu_apa = item_grouped_aggregated_sorted_eu_apa.groupby("ItemNumber").apply(calculate_pe)

In [ ]:
calculated_pe_eu = item_grouped_aggregated_sorted_eu.groupby("ItemNumber").apply(calculate_pe)

In [ ]:
calculated_pe_apa = item_grouped_aggregated_sorted_apa.groupby("ItemNumber").apply(calculate_pe)

In [ ]:
ungrouped_calculated_pe = calculated_pe.reset_index(drop=True)
ungrouped_calculated_pe.head(20)

In [ ]:
ungrouped_calculated_pe_usa = calculated_pe_usa.reset_index(drop=True)
ungrouped_calculated_pe_usa.head()

In [ ]:
ungrouped_calculated_pe_eu_apa = calculated_pe_eu_apa.reset_index(drop=True)
ungrouped_calculated_pe_eu_apa.head()

In [ ]:
ungrouped_calculated_pe_eu = calculated_pe_eu.reset_index(drop=True)
ungrouped_calculated_pe_apa = calculated_pe_apa.reset_index(drop=True)

In [ ]:
ungrouped_calculated_pe_eu.head(20)

# Checking distributions and clipping
Below we check price elasticity distributions. Most values are between -1 and 1, but there are a few outliers, which really influence the discovery process later on. For this reason, we clip the PE values at [-100, 100]. This is equivalent to a 2-3% clip on both sides.

In [ ]:


# Select the column of interest (assumed to be floats)
data = ungrouped_calculated_pe_eu_apa['InflationTimeAdjPriceElasticity']

# Print basic descriptive statistics using pandas' describe()
print("Descriptive Statistics:")
print(data.describe())

# Calculate mode (note: mode may return multiple values if there are ties)
mode_val = data.mode()
print("\nMode:")
print(mode_val)

# Also compute the median explicitly (even though it's in describe())
median_val = data.median()
print("\nMedian:", median_val)

In [ ]:
first_percentile = np.nanpercentile(ungrouped_calculated_pe_usa['InflationTimeAdjPriceElasticity'], 3)
print("First Percentile Value:", first_percentile)

In [ ]:
ungrouped_calculated_pe_eu_apa['ClippedInflationTimeAdjPriceElasticity'] = ungrouped_calculated_pe_eu_apa['InflationTimeAdjPriceElasticity'].clip(lower=-100, upper=100)

In [ ]:
ungrouped_calculated_pe_eu['ClippedInflationTimeAdjPriceElasticity'] = ungrouped_calculated_pe_eu['InflationTimeAdjPriceElasticity'].clip(lower=-100, upper=100)

In [ ]:
ungrouped_calculated_pe_apa['ClippedInflationTimeAdjPriceElasticity'] = ungrouped_calculated_pe_apa['InflationTimeAdjPriceElasticity'].clip(lower=-100, upper=100)

In [ ]:
ungrouped_calculated_pe_usa['ClippedInflationTimeAdjPriceElasticity'] = ungrouped_calculated_pe_usa['InflationTimeAdjPriceElasticity'].clip(lower=-100, upper=100)

In [ ]:
# Select the column of interest (assumed to be floats)
data = ungrouped_calculated_pe_eu_apa['ClippedInflationTimeAdjPriceElasticity']

# Print basic descriptive statistics using pandas' describe()
print("Descriptive Statistics:")
print(data.describe())

# Calculate mode (note: mode may return multiple values if there are ties)
mode_val = data.mode()
print("\nMode:")
print(mode_val)

# Also compute the median explicitly (even though it's in describe())
median_val = data.median()
print("\nMedian:", median_val)

In [ ]:
# Select the column of interest (assumed to be floats)
data = ungrouped_calculated_pe_usa['ClippedInflationTimeAdjPriceElasticity']

# Print basic descriptive statistics using pandas' describe()
print("Descriptive Statistics:")
print(data.describe())

# Calculate mode (note: mode may return multiple values if there are ties)
mode_val = data.mode()
print("\nMode:")
print(mode_val)

# Also compute the median explicitly (even though it's in describe())
median_val = data.median()
print("\nMedian:", median_val)

In [ ]:
# Select the column of interest (assumed to be floats)
data = ungrouped_calculated_pe_eu_apa['TimeAdjPriceElasticity']

# Print basic descriptive statistics using pandas' describe()
print("Descriptive Statistics:")
print(data.describe())

# Calculate mode (note: mode may return multiple values if there are ties)
mode_val = data.mode()
print("\nMode:")
print(mode_val)

# Also compute the median explicitly (even though it's in describe())
median_val = data.median()
print("\nMedian:", median_val)

# Aggregate to one row per product
Below, we get rid of redundant information. We have one row per product, containing all the features, and arrays where necessary.

In [ ]:
# Group by ItemNumber and aggregate the TimeAdjPriceElasticity and InflationTimeAdjPriceElasticity into arrays
aggregated_df = ungrouped_calculated_pe.groupby('ItemNumber').agg({
    'TimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'InflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'CurrentGLP': lambda x: x.dropna().tolist(),
    'DeflatedCurrentGLP': lambda x: x.dropna().tolist(),
    'CurrentQuantity': lambda x: x.dropna().tolist(),
    'CurrentDailyQty': lambda x: x.dropna().tolist(),
    'CurrentStartTime': lambda x: x.dropna().tolist(),
    'CurrentEndTime': lambda x: x.dropna().tolist(),
    'CurrentStartYear': lambda x: x.dropna().tolist(),
    'CurrentEndYear': lambda x: x.dropna().tolist(),
}).reset_index()

In [ ]:
aggregated_df.head()

In [ ]:
# Group by ItemNumber and aggregate the TimeAdjPriceElasticity and InflationTimeAdjPriceElasticity into arrays
aggregated_df_usa = ungrouped_calculated_pe_usa.groupby('ItemNumber').agg({
    'TimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'InflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'ClippedInflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'CurrentGLP': lambda x: x.dropna().tolist(),
    'DeflatedCurrentGLP': lambda x: x.dropna().tolist(),
    'CurrentQuantity': lambda x: x.dropna().tolist(),
    'CurrentDailyQty': lambda x: x.dropna().tolist(),
    'CurrentStartTime': lambda x: x.dropna().tolist(),
    'CurrentEndTime': lambda x: x.dropna().tolist(),
    'CurrentStartYear': lambda x: x.dropna().tolist(),
    'CurrentEndYear': lambda x: x.dropna().tolist(),
}).reset_index()

In [ ]:
aggregated_df_usa.head()

In [ ]:
# Group by ItemNumber and aggregate the TimeAdjPriceElasticity and InflationTimeAdjPriceElasticity into arrays
aggregated_df_eu_apa = ungrouped_calculated_pe_eu_apa.groupby('ItemNumber').agg({
    'TimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'InflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'ClippedInflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'CurrentGLP': lambda x: x.dropna().tolist(),
    'DeflatedCurrentGLP': lambda x: x.dropna().tolist(),
    'CurrentQuantity': lambda x: x.dropna().tolist(),
    'CurrentDailyQty': lambda x: x.dropna().tolist(),
    'CurrentStartTime': lambda x: x.dropna().tolist(),
    'CurrentEndTime': lambda x: x.dropna().tolist(),
    'CurrentStartYear': lambda x: x.dropna().tolist(),
    'CurrentEndYear': lambda x: x.dropna().tolist(),
}).reset_index()

In [ ]:
aggregated_df_eu_apa.head()

In [ ]:
# Group by ItemNumber and aggregate the TimeAdjPriceElasticity and InflationTimeAdjPriceElasticity into arrays
aggregated_df_eu = ungrouped_calculated_pe_eu.groupby('ItemNumber').agg({
    'TimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'InflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'ClippedInflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'CurrentGLP': lambda x: x.dropna().tolist(),
    'DeflatedCurrentGLP': lambda x: x.dropna().tolist(),
    'CurrentQuantity': lambda x: x.dropna().tolist(),
    'CurrentDailyQty': lambda x: x.dropna().tolist(),
    'CurrentStartTime': lambda x: x.dropna().tolist(),
    'CurrentEndTime': lambda x: x.dropna().tolist(),
    'CurrentStartYear': lambda x: x.dropna().tolist(),
    'CurrentEndYear': lambda x: x.dropna().tolist(),
}).reset_index()

In [ ]:
aggregated_df_eu.head()

In [ ]:
# Group by ItemNumber and aggregate the TimeAdjPriceElasticity and InflationTimeAdjPriceElasticity into arrays
aggregated_df_apa = ungrouped_calculated_pe_apa.groupby('ItemNumber').agg({
    'TimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'InflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'ClippedInflationTimeAdjPriceElasticity': lambda x: x.dropna().tolist(),
    'CurrentGLP': lambda x: x.dropna().tolist(),
    'DeflatedCurrentGLP': lambda x: x.dropna().tolist(),
    'CurrentQuantity': lambda x: x.dropna().tolist(),
    'CurrentDailyQty': lambda x: x.dropna().tolist(),
    'CurrentStartTime': lambda x: x.dropna().tolist(),
    'CurrentEndTime': lambda x: x.dropna().tolist(),
    'CurrentStartYear': lambda x: x.dropna().tolist(),
    'CurrentEndYear': lambda x: x.dropna().tolist(),
}).reset_index()

In [ ]:
aggregated_df_apa.head()

In [ ]:
aggregated_df.info()

In [ ]:
ungrouped_calculated_pe.info()

In [ ]:
# Select the columns to merge from calculated_pe
columns_to_merge = ['ItemNumber', 'MeanSalesPrice', 'MeanCostPrice', 'MeanManualDiscount',  'MeanSupplierLeadtime', 'PriceCategory', 'PercentEU', 'PercentUSA', 'PercentAPA', 'VIEngineered', 'SparePartsCategory', 'TechnicalClassification', 'OEM', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5']

# Merge the selected columns into aggregated_df
aggregated_df = pd.merge(aggregated_df, ungrouped_calculated_pe[columns_to_merge].drop_duplicates(subset=['ItemNumber']), on='ItemNumber', how='left')

In [ ]:
aggregated_df_usa = pd.merge(aggregated_df_usa, ungrouped_calculated_pe_usa[columns_to_merge].drop_duplicates(subset=['ItemNumber']), on='ItemNumber', how='left')

In [ ]:
aggregated_df_eu_apa = pd.merge(aggregated_df_eu_apa, ungrouped_calculated_pe_eu_apa[columns_to_merge].drop_duplicates(subset=['ItemNumber']), on='ItemNumber', how='left')

In [ ]:
aggregated_df_eu = pd.merge(aggregated_df_eu, ungrouped_calculated_pe_eu[columns_to_merge].drop_duplicates(subset=['ItemNumber']), on='ItemNumber', how='left')

In [ ]:
aggregated_df_apa = pd.merge(aggregated_df_apa, ungrouped_calculated_pe_apa[columns_to_merge].drop_duplicates(subset=['ItemNumber']), on='ItemNumber', how='left')

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
aggregated_df.head()

In [ ]:
aggregated_df_usa.head()

In [ ]:
aggregated_df_eu_apa.head()

In [ ]:
aggregated_df['MeanProfit'] = np.where((aggregated_df['MeanSalesPrice'] == 0) | (aggregated_df['MeanCostPrice'] == 0), np.nan, aggregated_df['MeanSalesPrice'] - aggregated_df['MeanCostPrice'])

In [ ]:
aggregated_df_usa['MeanProfit'] = np.where((aggregated_df_usa['MeanSalesPrice'] == 0) | (aggregated_df_usa['MeanCostPrice'] == 0), np.nan, aggregated_df_usa['MeanSalesPrice'] - aggregated_df_usa['MeanCostPrice'])


In [ ]:
aggregated_df_eu_apa['MeanProfit'] = np.where((aggregated_df_eu_apa['MeanSalesPrice'] == 0) | (aggregated_df_eu_apa['MeanCostPrice'] == 0), np.nan, aggregated_df_eu_apa['MeanSalesPrice'] - aggregated_df_eu_apa['MeanCostPrice'])

In [ ]:
aggregated_df_eu['MeanProfit'] = np.where((aggregated_df_eu['MeanSalesPrice'] == 0) | (aggregated_df_eu['MeanCostPrice'] == 0), np.nan, aggregated_df_eu['MeanSalesPrice'] - aggregated_df_eu['MeanCostPrice'])

In [ ]:
aggregated_df_apa['MeanProfit'] = np.where((aggregated_df_apa['MeanSalesPrice'] == 0) | (aggregated_df_apa['MeanCostPrice'] == 0), np.nan, aggregated_df_apa['MeanSalesPrice'] - aggregated_df_apa['MeanCostPrice'])

# Compute trends for categorical target

In [ ]:
def has_zero_to_neg_transition(array):
    """
    Checks if the array has a transition from zero to negative values
    without returning to zero or positive.
    """
    seen_negative = False  # Tracks if we have encountered negative numbers
    for value in array:
        if value == 0:
            if seen_negative:
                return False  # Encountered a zero after a negative
        elif value < 0:
            seen_negative = True  # Start tracking negative numbers
        else:
            return False  # Positive values are not allowed
    # Ensure the array has at least one zero and one negative number
    return seen_negative and array[0] == 0

def has_zero_to_pos_transition(array):
    """
    Checks if the array has a transition from zero to positive values
    without returning to zero or negative.
    """
    seen_positive = False  # Tracks if we have encountered positive numbers
    for value in array:
        if value == 0:
            if seen_positive:
                return False  # Encountered a zero after a positive
        elif value > 0:
            seen_positive = True  # Start tracking positive numbers
        else:
            return False  # Negative values are not allowed
    # Ensure the array has at least one zero and one positive number
    return seen_positive and array[0] == 0
def has_negative_to_positive_transition(array):
    """
    Checks if the array has a transition from negative to positive values
    without returning to negative.
    """
    seen_positive = False  # Tracks if we have encountered positive numbers
    for value in array:
        if value < 0:
            if seen_positive:
                return False  # Encountered a negative number after a positive
        elif value > 0:
            seen_positive = True  # Start tracking positive numbers
        else:
            return False  # Zero is not allowed
    # Ensure the array has at least one negative and one positive number
    return seen_positive and array[0] < 0

def has_positive_to_negative_transition(array):
    """
    Checks if the array has a transition from negative to positive values
    without returning to negative.
    """
    seen_negative = False  # Tracks if we have encountered negative numbers
    for value in array:
        if value > 0:
            if seen_negative:
                return False  # Encountered a positive number after a negative
        elif value < 0:
            seen_negative = True  # Start tracking negative numbers
        else:
            return False  # Zero is not allowed
    # Ensure the array has at least one positive and one negative number
    return seen_negative and array[0] > 0

def is_positive_then_zero(array):
    """
    Checks if the array starts with n positive numbers followed by m zeroes (n > 0, m > 0).
    The array must start with at least one positive number and transition to zeroes without returning to positive or encountering negative values.
    """
    seen_zero = False  # Tracks if we have encountered zeroes
    for value in array:
        if value > 0:
            if seen_zero:
                return False  # Encountered a positive number after zeroes
        elif value == 0:
            seen_zero = True  # Start tracking zeroes
        else:
            return False  # Negative values are not allowed
    # Ensure the array has at least one positive number and one zero
    return seen_zero and array[0] > 0

def is_negative_then_zero(array):

    seen_zero = False  # Tracks if we have encountered zeroes
    for value in array:
        if value < 0:
            if seen_zero:
                return False  # Encountered a positive number after zeroes
        elif value == 0:
            seen_zero = True  # Start tracking zeroes
        else:
            return False  # Negative values are not allowed
    # Ensure the array has at least one positive number and one zero
    return seen_zero and array[0] < 0
# Define the trend function for negative to zero transition
def detect_price_elasticity_trend(row):
    if is_negative_then_zero(row['TimeAdjPriceElasticity']):
        return "neg to zero"
    if is_positive_then_zero(row['TimeAdjPriceElasticity']):
        return "pos to zero"
    if has_positive_to_negative_transition(row['TimeAdjPriceElasticity']):
        return "pos to neg"
    if has_negative_to_positive_transition(row['TimeAdjPriceElasticity']):
        return "neg to pos"
    if has_zero_to_neg_transition(row['TimeAdjPriceElasticity']):
        return "zero to neg"
    if has_zero_to_pos_transition(row['TimeAdjPriceElasticity']):
        return "zero to pos"
    if all(e > 0 for e in row['TimeAdjPriceElasticity']):
        return "positive"
    if all(e < 0 for e in row['TimeAdjPriceElasticity']):
        return "negative"
    if all(e == 0 for e in row['TimeAdjPriceElasticity']):
        return "zero"
    return "erratic"  # Default for rows not matching the condition

# Add the PriceElasticityTrend column
aggregated_df['PriceElasticityTrendTimeAdj'] = aggregated_df.apply(detect_price_elasticity_trend, axis=1)
# Define the trend function for negative to zero transition
def detect_price_elasticity_trend_inflation(row):
    if is_negative_then_zero(row['InflationTimeAdjPriceElasticity']):
        return "neg to zero"
    if is_positive_then_zero(row['InflationTimeAdjPriceElasticity']):
        return "pos to zero"
    if has_positive_to_negative_transition(row['InflationTimeAdjPriceElasticity']):
        return "pos to neg"
    if has_negative_to_positive_transition(row['InflationTimeAdjPriceElasticity']):
        return "neg to pos"
    if has_zero_to_neg_transition(row['InflationTimeAdjPriceElasticity']):
        return "zero to neg"
    if has_zero_to_pos_transition(row['InflationTimeAdjPriceElasticity']):
        return "zero to pos"
    if all(e > 0 for e in row['InflationTimeAdjPriceElasticity']):
        return "positive"
    if all(e < 0 for e in row['InflationTimeAdjPriceElasticity']):
        return "negative"
    if all(e == 0 for e in row['InflationTimeAdjPriceElasticity']):
        return "zero"
    return "erratic"  # Default for rows not matching the condition

# Add the PriceElasticityTrend column
aggregated_df['InflPriceElasticityTrendTimeAdj'] = aggregated_df.apply(detect_price_elasticity_trend_inflation, axis=1)

In [ ]:
aggregated_df_usa['PriceElasticityTrendTimeAdj'] = aggregated_df_usa.apply(detect_price_elasticity_trend, axis=1)
aggregated_df_usa['InflPriceElasticityTrendTimeAdj'] = aggregated_df_usa.apply(detect_price_elasticity_trend_inflation, axis=1)

In [ ]:
aggregated_df_eu_apa['PriceElasticityTrendTimeAdj'] = aggregated_df_eu_apa.apply(detect_price_elasticity_trend, axis=1)
aggregated_df_eu_apa['InflPriceElasticityTrendTimeAdj'] = aggregated_df_eu_apa.apply(detect_price_elasticity_trend_inflation, axis=1)

In [ ]:
aggregated_df_eu['PriceElasticityTrendTimeAdj'] = aggregated_df_eu.apply(detect_price_elasticity_trend, axis=1)
aggregated_df_eu['InflPriceElasticityTrendTimeAdj'] = aggregated_df_eu.apply(detect_price_elasticity_trend_inflation, axis=1)

In [ ]:
aggregated_df_apa['PriceElasticityTrendTimeAdj'] = aggregated_df_apa.apply(detect_price_elasticity_trend, axis=1)
aggregated_df_apa['InflPriceElasticityTrendTimeAdj'] = aggregated_df_apa.apply(detect_price_elasticity_trend_inflation, axis=1)

In [ ]:
aggregated_df.head()

In [ ]:

aggregated_df_eu_apa.head()

In [ ]:
aggregated_df_usa.head()

In [ ]:
aggregated_df['PriceElasticityTrendTimeAdj'].value_counts()

In [ ]:
aggregated_df['InflPriceElasticityTrendTimeAdj'].value_counts()

In [ ]:
aggregated_df_usa['InflPriceElasticityTrendTimeAdj'].value_counts()

In [ ]:
aggregated_df_eu_apa['InflPriceElasticityTrendTimeAdj'].value_counts()

In [ ]:
mean_order_qty = df_wide_data.groupby('ItemNumber')['OrderQuantity'].mean().reset_index()
mean_order_qty.columns = ['ItemNumber', 'MeanOrderQty']

In [ ]:
mean_order_qty_usa = df_wide_data_usa.groupby('ItemNumber')['OrderQuantity'].mean().reset_index()
mean_order_qty_usa.columns = ['ItemNumber', 'MeanOrderQty']
aggregated_df_usa = pd.merge(aggregated_df_usa, mean_order_qty_usa, on='ItemNumber', how='left')

In [ ]:
mean_order_qty_eu_apa = df_wide_data_eu_apa.groupby('ItemNumber')['OrderQuantity'].mean().reset_index()
mean_order_qty_eu_apa.columns = ['ItemNumber', 'MeanOrderQty']
aggregated_df_eu_apa = pd.merge(aggregated_df_eu_apa, mean_order_qty_eu_apa, on='ItemNumber', how='left')

In [ ]:
mean_order_qty_eu = df_wide_data_eu.groupby('ItemNumber')['OrderQuantity'].mean().reset_index()
mean_order_qty_eu.columns = ['ItemNumber', 'MeanOrderQty']
aggregated_df_eu = pd.merge(aggregated_df_eu, mean_order_qty_eu, on='ItemNumber', how='left')

In [ ]:
mean_order_qty_apa = df_wide_data_apa.groupby('ItemNumber')['OrderQuantity'].mean().reset_index()
mean_order_qty_apa.columns = ['ItemNumber', 'MeanOrderQty']
aggregated_df_apa = pd.merge(aggregated_df_apa, mean_order_qty_apa, on='ItemNumber', how='left')

In [ ]:
aggregated_df = pd.merge(aggregated_df, mean_order_qty, on='ItemNumber', how='left')

In [ ]:

aggregated_df.head()

In [ ]:
aggregated_df.info()

In [ ]:
df_wide_data.info()

# Compute new features

In [ ]:
mean_discount = df_wide_data.groupby('ItemNumber')['Discount'].mean().reset_index()
mean_discount.columns = ['ItemNumber', 'Discount']

# Merge the mean order quantity into emm_data_df
aggregated_df_new = pd.merge(aggregated_df, mean_discount, on='ItemNumber', how='left')

In [ ]:
aggregated_df_new.head()

In [ ]:
mean_discount_usa = df_wide_data_usa.groupby('ItemNumber')['Discount'].mean().reset_index()
mean_discount_usa.columns = ['ItemNumber', 'Discount']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_usa = pd.merge(aggregated_df_usa, mean_discount_usa, on='ItemNumber', how='left')

In [ ]:
mean_discount_eu_apa = df_wide_data_eu_apa.groupby('ItemNumber')['Discount'].mean().reset_index()
mean_discount_eu_apa.columns = ['ItemNumber', 'Discount']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu_apa = pd.merge(aggregated_df_eu_apa, mean_discount_eu_apa, on='ItemNumber', how='left')

In [ ]:
mean_discount_eu = df_wide_data_eu.groupby('ItemNumber')['Discount'].mean().reset_index()
mean_discount_eu.columns = ['ItemNumber', 'Discount']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu = pd.merge(aggregated_df_eu, mean_discount_eu, on='ItemNumber', how='left')

In [ ]:
mean_discount_apa = df_wide_data_apa.groupby('ItemNumber')['Discount'].mean().reset_index()
mean_discount_apa.columns = ['ItemNumber', 'Discount']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_apa = pd.merge(aggregated_df_apa, mean_discount_apa, on='ItemNumber', how='left')

In [ ]:
items_in_order = df_wide_data.groupby(['OrderDate', 'CustomerSoldTo'])['ItemNumber'].nunique().reset_index()
df_wide_data_items_in_order = df_wide_data.merge(
    items_in_order,
    on=['OrderDate', 'CustomerSoldTo'],
    how='left',
    suffixes=('', '_ItemsInOrder')
)

In [ ]:
df_wide_data_items_in_order.rename(columns={'ItemNumber_ItemsInOrder': 'ItemsInOrder'}, inplace=True)

In [ ]:
items_in_order_usa = df_wide_data_usa.groupby(['OrderDate', 'CustomerSoldTo'])['ItemNumber'].nunique().reset_index()
df_wide_data_items_in_order_usa = df_wide_data_usa.merge(
    items_in_order_usa,
    on=['OrderDate', 'CustomerSoldTo'],
    how='left',
    suffixes=('', '_ItemsInOrder')
)
df_wide_data_items_in_order_usa.rename(columns={'ItemNumber_ItemsInOrder': 'ItemsInOrder'}, inplace=True)

In [ ]:
items_in_order_eu_apa = df_wide_data_eu_apa.groupby(['OrderDate', 'CustomerSoldTo'])['ItemNumber'].nunique().reset_index()
df_wide_data_items_in_order_eu_apa = df_wide_data_eu_apa.merge(
    items_in_order_eu_apa,
    on=['OrderDate', 'CustomerSoldTo'],
    how='left',
    suffixes=('', '_ItemsInOrder')
)
df_wide_data_items_in_order_eu_apa.rename(columns={'ItemNumber_ItemsInOrder': 'ItemsInOrder'}, inplace=True)

In [ ]:
items_in_order_eu = df_wide_data_eu.groupby(['OrderDate', 'CustomerSoldTo'])['ItemNumber'].nunique().reset_index()
df_wide_data_items_in_order_eu = df_wide_data_eu.merge(
    items_in_order_eu,
    on=['OrderDate', 'CustomerSoldTo'],
    how='left',
    suffixes=('', '_ItemsInOrder')
)
df_wide_data_items_in_order_eu.rename(columns={'ItemNumber_ItemsInOrder': 'ItemsInOrder'}, inplace=True)

In [ ]:
items_in_order_apa = df_wide_data_apa.groupby(['OrderDate', 'CustomerSoldTo'])['ItemNumber'].nunique().reset_index()
df_wide_data_items_in_order_apa = df_wide_data_apa.merge(
    items_in_order_apa,
    on=['OrderDate', 'CustomerSoldTo'],
    how='left',
    suffixes=('', '_ItemsInOrder')
)
df_wide_data_items_in_order_apa.rename(columns={'ItemNumber_ItemsInOrder': 'ItemsInOrder'}, inplace=True)

In [ ]:
df_wide_data_items_in_order.head()

In [ ]:
mean_items_in_order = df_wide_data_items_in_order.groupby('ItemNumber')['ItemsInOrder'].mean().reset_index()
mean_items_in_order.columns = ['ItemNumber', 'ItemsInOrder']

# Merge the mean order quantity into emm_data_df
aggregated_df_new = pd.merge(aggregated_df_new, mean_items_in_order, on='ItemNumber', how='left')

In [ ]:
mean_items_in_order_usa = df_wide_data_items_in_order_usa.groupby('ItemNumber')['ItemsInOrder'].mean().reset_index()
mean_items_in_order_usa.columns = ['ItemNumber', 'ItemsInOrder']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_usa = pd.merge(aggregated_df_new_usa, mean_items_in_order_usa, on='ItemNumber', how='left')

In [ ]:
mean_items_in_order_eu_apa = df_wide_data_items_in_order_eu_apa.groupby('ItemNumber')['ItemsInOrder'].mean().reset_index()
mean_items_in_order_eu_apa.columns = ['ItemNumber', 'ItemsInOrder']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu_apa = pd.merge(aggregated_df_new_eu_apa, mean_items_in_order_eu_apa, on='ItemNumber', how='left')

In [ ]:
mean_items_in_order_eu = df_wide_data_items_in_order_eu.groupby('ItemNumber')['ItemsInOrder'].mean().reset_index()
mean_items_in_order_eu.columns = ['ItemNumber', 'ItemsInOrder']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu = pd.merge(aggregated_df_new_eu, mean_items_in_order_eu, on='ItemNumber', how='left')

In [ ]:
mean_items_in_order_apa = df_wide_data_items_in_order_apa.groupby('ItemNumber')['ItemsInOrder'].mean().reset_index()
mean_items_in_order_apa.columns = ['ItemNumber', 'ItemsInOrder']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_apa = pd.merge(aggregated_df_new_apa, mean_items_in_order_apa, on='ItemNumber', how='left')

In [ ]:
aggregated_df_new.head()

In [ ]:
aggregated_df_new_usa.head()

In [ ]:
aggregated_df_new_eu_apa.head()

In [ ]:
# Filter the DataFrame and select the top 100 rows
#price_input_error = df_wide_data_items_in_order[df_wide_data_items_in_order['PriceSalesUoMEUR'] >= 100000].head(100)

# Save the filtered DataFrame to an Excel file
#price_input_error.to_excel('price_input_error.xlsx', index=False)

In [ ]:
df_wide_data_items_in_order['DiscountPercent'] = (df_wide_data_items_in_order['GLPSalesUoM'] - df_wide_data_items_in_order['PriceSalesUoMEUR']) / df_wide_data_items_in_order['GLPSalesUoM']

df_wide_data_items_in_order.loc[df_wide_data_items_in_order['GLPSalesUoM'] == 0, 'DiscountPercent'] = 0
df_wide_data_items_in_order.loc[df_wide_data_items_in_order['PriceSalesUoMEUR'] == 0, 'DiscountPercent'] = 0

mean_discount_percent = df_wide_data_items_in_order.groupby('ItemNumber')['DiscountPercent'].mean().reset_index()
mean_discount_percent.columns = ['ItemNumber', 'DiscountPercent']

# Merge the mean order quantity into emm_data_df
aggregated_df_new = pd.merge(aggregated_df_new, mean_discount_percent, on='ItemNumber', how='left')

In [ ]:
df_wide_data_items_in_order_usa['DiscountPercent'] = (df_wide_data_items_in_order_usa['GLPSalesUoM'] - df_wide_data_items_in_order_usa['PriceSalesUoMEUR']) / df_wide_data_items_in_order_usa['GLPSalesUoM']

df_wide_data_items_in_order_usa.loc[df_wide_data_items_in_order_usa['GLPSalesUoM'] == 0, 'DiscountPercent'] = 0
df_wide_data_items_in_order_usa.loc[df_wide_data_items_in_order_usa['PriceSalesUoMEUR'] == 0, 'DiscountPercent'] = 0

mean_discount_percent_usa = df_wide_data_items_in_order_usa.groupby('ItemNumber')['DiscountPercent'].mean().reset_index()
mean_discount_percent_usa.columns = ['ItemNumber', 'DiscountPercent']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_usa = pd.merge(aggregated_df_new_usa, mean_discount_percent_usa, on='ItemNumber', how='left')

In [ ]:
df_wide_data_items_in_order_eu_apa['DiscountPercent'] = (df_wide_data_items_in_order_eu_apa['GLPSalesUoM'] - df_wide_data_items_in_order_eu_apa['PriceSalesUoMEUR']) / df_wide_data_items_in_order_eu_apa['GLPSalesUoM']

df_wide_data_items_in_order_eu_apa.loc[df_wide_data_items_in_order_eu_apa['GLPSalesUoM'] == 0, 'DiscountPercent'] = 0
df_wide_data_items_in_order_eu_apa.loc[df_wide_data_items_in_order_eu_apa['PriceSalesUoMEUR'] == 0, 'DiscountPercent'] = 0

mean_discount_percent_eu_apa = df_wide_data_items_in_order_eu_apa.groupby('ItemNumber')['DiscountPercent'].mean().reset_index()
mean_discount_percent_eu_apa.columns = ['ItemNumber', 'DiscountPercent']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu_apa = pd.merge(aggregated_df_new_eu_apa, mean_discount_percent_eu_apa, on='ItemNumber', how='left')

In [ ]:
df_wide_data_items_in_order_eu['DiscountPercent'] = (df_wide_data_items_in_order_eu['GLPSalesUoM'] - df_wide_data_items_in_order_eu['PriceSalesUoMEUR']) / df_wide_data_items_in_order_eu['GLPSalesUoM']

df_wide_data_items_in_order_eu.loc[df_wide_data_items_in_order_eu['GLPSalesUoM'] == 0, 'DiscountPercent'] = 0
df_wide_data_items_in_order_eu.loc[df_wide_data_items_in_order_eu['PriceSalesUoMEUR'] == 0, 'DiscountPercent'] = 0

mean_discount_percent_eu = df_wide_data_items_in_order_eu.groupby('ItemNumber')['DiscountPercent'].mean().reset_index()
mean_discount_percent_eu.columns = ['ItemNumber', 'DiscountPercent']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu = pd.merge(aggregated_df_new_eu, mean_discount_percent_eu, on='ItemNumber', how='left')

In [ ]:
df_wide_data_items_in_order_apa['DiscountPercent'] = (df_wide_data_items_in_order_apa['GLPSalesUoM'] - df_wide_data_items_in_order_apa['PriceSalesUoMEUR']) / df_wide_data_items_in_order_apa['GLPSalesUoM']

df_wide_data_items_in_order_apa.loc[df_wide_data_items_in_order_apa['GLPSalesUoM'] == 0, 'DiscountPercent'] = 0
df_wide_data_items_in_order_apa.loc[df_wide_data_items_in_order_apa['PriceSalesUoMEUR'] == 0, 'DiscountPercent'] = 0

mean_discount_percent_apa = df_wide_data_items_in_order_apa.groupby('ItemNumber')['DiscountPercent'].mean().reset_index()
mean_discount_percent_apa.columns = ['ItemNumber', 'DiscountPercent']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_apa = pd.merge(aggregated_df_new_apa, mean_discount_percent_apa, on='ItemNumber', how='left')

In [ ]:
StockedItems = pd.read_excel("Stocked Items.xlsx", sheet_name = "S4Export_12")
# Create a set of stocked item codes for faster lookup
stocked_item_codes = set(StockedItems['Code'])


In [ ]:

# Add the 'stocked' column to merged_data
aggregated_df_new['Stocked'] = aggregated_df_new['ItemNumber'].apply(lambda x: 1 if x in stocked_item_codes else 0)

In [ ]:
aggregated_df_new_eu_apa['Stocked'] = aggregated_df_new_eu_apa['ItemNumber'].apply(lambda x: 1 if x in stocked_item_codes else 0)

In [ ]:
aggregated_df_new_eu['Stocked'] = aggregated_df_new_eu['ItemNumber'].apply(lambda x: 1 if x in stocked_item_codes else 0)

In [ ]:
aggregated_df_new_apa['Stocked'] = aggregated_df_new_apa['ItemNumber'].apply(lambda x: 1 if x in stocked_item_codes else 0)

In [ ]:
aggregated_df_new_usa['Stocked'] = aggregated_df_new_usa['ItemNumber'].apply(lambda x: 1 if x in stocked_item_codes else 0)

In [ ]:
aggregated_df_new.head()

In [ ]:
len(aggregated_df_new)

In [ ]:
len(aggregated_df_new_usa)

In [ ]:
len(aggregated_df_new_eu_apa)

In [ ]:
# Count the number of rows where PriceElasticityTrendTimeAdj is not equal to InflPriceElasticityTrendTimeAdj
mismatch_count = aggregated_df_new[aggregated_df_new['PriceElasticityTrendTimeAdj'] != aggregated_df_new['InflPriceElasticityTrendTimeAdj']].shape[0]
print(f'There are {mismatch_count} rows where PriceElasticityTrendTimeAdj is not equal to InflPriceElasticityTrendTimeAdj.')

In [ ]:
aggregated_df_new.head()

# Resample to 8 elasticities per product
Here we resample to 8 elasticities per product, roughly corresponding to 2 price changes a year over 4 years. If you want to run on a different period, changed fixed length to the number of your choice.

In [ ]:
def resample_array(arr, fixed_length):
    """
    Resample a 1D numpy array to a fixed length using linear interpolation.

    Parameters:
        arr (array-like): Original array of numeric values.
        fixed_length (int): Desired length of the resampled array.

    Returns:
        numpy.ndarray: Resampled array of length 'fixed_length'.
    """
    arr = np.asarray(arr, dtype=float)
    original_length = len(arr)

    if original_length == fixed_length:
        return arr

    # Create original and new equally spaced indices (scaled from 0 to 1)
    original_indices = np.linspace(0, 1, original_length)
    target_indices = np.linspace(0, 1, fixed_length)

    # Use linear interpolation to compute resampled values
    resampled = np.interp(target_indices, original_indices, arr)

    return resampled


In [ ]:
# Define the fixed length you want
fixed_length = 8

# Use your resample_array function
aggregated_df_new['ResampledInflationPE'] = aggregated_df_new['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_usa['ResampledInflationPE'] = aggregated_df_new_usa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['ResampledInflationPE'] = aggregated_df_new_eu_apa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
# Use your resample_array function
aggregated_df_new['ResampledStandardPE'] = aggregated_df_new['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_usa['ResampledStandardPE'] = aggregated_df_new_usa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['ResampledStandardPE'] = aggregated_df_new_eu_apa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
# Define the fixed length you want
fixed_length = 8


aggregated_df_new_usa['ClippedResampledInflationPE'] = aggregated_df_new_usa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['ClippedResampledInflationPE'] = aggregated_df_new_eu_apa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['ResampledInflationPE'] = aggregated_df_new_eu['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['ResampledStandardPE'] = aggregated_df_new_eu['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['ClippedResampledInflationPE'] = aggregated_df_new_eu['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['ResampledInflationPE'] = aggregated_df_new_apa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['ResampledStandardPE'] = aggregated_df_new_apa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['ClippedResampledInflationPE'] = aggregated_df_new_apa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))

In [ ]:
# Define the fixed length you want
fixed_length = 8

# Use your resample_array function
aggregated_df_new['8ResampledInflationPE'] = aggregated_df_new['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_usa['8ResampledInflationPE'] = aggregated_df_new_usa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['8ResampledInflationPE'] = aggregated_df_new_eu_apa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
# Use your resample_array function
aggregated_df_new['8ResampledStandardPE'] = aggregated_df_new['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_usa['8ResampledStandardPE'] = aggregated_df_new_usa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['8ResampledStandardPE'] = aggregated_df_new_eu_apa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))



aggregated_df_new_usa['8ClippedResampledInflationPE'] = aggregated_df_new_usa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu_apa['8ClippedResampledInflationPE'] = aggregated_df_new_eu_apa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['8ResampledInflationPE'] = aggregated_df_new_eu['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['8ResampledStandardPE'] = aggregated_df_new_eu['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_eu['8ClippedResampledInflationPE'] = aggregated_df_new_eu['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['8ResampledInflationPE'] = aggregated_df_new_apa['InflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['8ResampledStandardPE'] = aggregated_df_new_apa['TimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))
aggregated_df_new_apa['8ClippedResampledInflationPE'] = aggregated_df_new_apa['ClippedInflationTimeAdjPriceElasticity'].apply(
    lambda arr: resample_array(np.array(arr), fixed_length))

In [ ]:
aggregated_df_new.head()

# add consolidation customers

In [ ]:

# Load consolidation customers from the specified Excel file
consolidation_customers = pd.read_excel("ConsolidationCustomers.xlsx")

# Display the first few rows of the dataframe

consolidation_customers.rename(columns={'Ship to:': 'ShipTo'}, inplace=True)

consolidation_customers.rename(columns={'Consignee ': 'Consignee'}, inplace=True)
consolidation_customers['ShipTo'] = consolidation_customers['ShipTo'].astype("string")
consolidation_customers['Consignee'] = consolidation_customers['Consignee'].astype("string")
consolidation_customers['City'] = consolidation_customers['City'].astype("string")
consolidation_customers['Country'] = consolidation_customers['Country'].astype("string")
def split_customer_sold_to(row):
    """
    Splits 'CustomerSoldTo' into CustomerNumber and CustomerName.
    - Extracts the first numeric part as CustomerNumber.
    - The remainder of the string is treated as CustomerName.
    - If no number or name exists, returns None for that part.
    """
    if pd.isnull(row):
        return pd.Series({'CustomerNumber': None, 'CustomerName': None})
    
    parts = row.split(' - ', 1)  # Split at first occurrence
    if len(parts) == 2:
        number_part, name_part = parts[0].strip(), parts[1].strip()
        return pd.Series({'CustomerNumber': number_part, 'CustomerName': name_part})
    else:
        # Handle cases without exactly one ' - '
        number_part = ''.join(filter(str.isdigit, parts[0])).strip()
        name_part = parts[0].strip() if number_part == '' else None
        return pd.Series({'CustomerNumber': number_part if number_part else None,
                          'CustomerName': name_part if len(parts) > 1 else name_part})
df_wide_data_items_in_order_copy = df_wide_data_items_in_order.copy()
# Apply function:
df_wide_data_items_in_order_copy[['CustomerNumber', 'CustomerName']] = df_wide_data_items_in_order_copy['CustomerSoldTo'].apply(split_customer_sold_to)

def split_ship_to(row):
    """
    Splits 'CustomerSoldTo' into CustomerNumber and CustomerName.
    - Extracts the first numeric part as CustomerNumber.
    - The remainder of the string is treated as CustomerName.
    - If no number or name exists, returns None for that part.
    """
    if pd.isnull(row):
        return pd.Series({'ShipNumber': None, 'ShipName': None})
    
    parts = row.split(' - ', 1)  # Split at first occurrence
    if len(parts) == 2:
        number_part, name_part = parts[0].strip(), parts[1].strip()
        return pd.Series({'ShipNumber': number_part, 'ShipName': name_part})
    else:
        # Handle cases without exactly one ' - '
        number_part = ''.join(filter(str.isdigit, parts[0])).strip()
        name_part = parts[0].strip() if number_part == '' else None
        return pd.Series({'ShipNumber': number_part if number_part else None,
                          'ShipName': name_part if len(parts) > 1 else name_part})
    
df_wide_data_items_in_order_copy[['ShipNumber', 'ShipName']] = df_wide_data_items_in_order_copy['ShipTo'].apply(split_ship_to)
df_wide_data_items_in_order_copy['ShipNumber'] = df_wide_data_items_in_order_copy['ShipNumber'].str.replace(' ', '', regex=False)
df_wide_data_items_in_order_copy['CustomerNumber'] = df_wide_data_items_in_order_copy['CustomerNumber'].str.replace(' ', '', regex=False)
consolidation_customers['ShipTo'] = consolidation_customers['ShipTo'].str.replace(' ', '', regex=False)

consignee_list = consolidation_customers['Consignee'].dropna().unique().tolist()

# Define a helper function to match CustomerName with Consignee strings
def consignee_match(customer_name, consignee_list):
    if pd.isnull(customer_name):
        return False
    for consignee in consignee_list:
        if consignee.lower() in customer_name.lower():
            return True
    return False

# Apply conditions
df_wide_data_items_in_order_copy['ConsolidationCustomer'] = np.where(
    df_wide_data_items_in_order_copy['ShipNumber'].isin(consolidation_customers['ShipTo']) |
    df_wide_data_items_in_order_copy['CustomerNumber'].isin(consolidation_customers['ShipTo']) 
    #| df_wide_data_items_in_order_copy['CustomerName'].apply(lambda x: consignee_match(x, consolidation_customers['Consignee '])) |
    #df_wide_data_items_in_order_copy['ShipName'].apply(lambda x: consignee_match(x, consolidation_customers['Consignee ']))
    
    , 1, 0)





In [ ]:
df_wide_data_items_in_order_copy['ConsolidationCustomer'].value_counts()

In [ ]:
df_wide_data_items_in_order = df_wide_data_items_in_order_copy

In [ ]:
df_wide_data_items_in_order.head()

In [ ]:
df_wide_data_items_in_order_eu_apa = df_wide_data_items_in_order[df_wide_data_items_in_order['GeographicRegion'] != 'USA']

In [ ]:
df_wide_data_items_in_order_eu = df_wide_data_items_in_order[df_wide_data_items_in_order['GeographicRegion'] == 'EUR']

In [ ]:
df_wide_data_items_in_order_apa = df_wide_data_items_in_order[df_wide_data_items_in_order['GeographicRegion'] == 'APA']

In [ ]:
df_wide_data_items_in_order_usa = df_wide_data_items_in_order[df_wide_data_items_in_order['GeographicRegion'] == 'USA']

In [ ]:
perc_consolidation_cust_eu_apa = df_wide_data_items_in_order_eu_apa.groupby('ItemNumber')['ConsolidationCustomer'].mean().reset_index()
perc_consolidation_cust_eu_apa.columns = ['ItemNumber', 'ConsolidationCustomer']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu_apa = pd.merge(aggregated_df_new_eu_apa, perc_consolidation_cust_eu_apa, on='ItemNumber', how='left')

In [ ]:
perc_consolidation_cust_eu = df_wide_data_items_in_order_eu.groupby('ItemNumber')['ConsolidationCustomer'].mean().reset_index()
perc_consolidation_cust_eu.columns = ['ItemNumber', 'ConsolidationCustomer']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_eu = pd.merge(aggregated_df_new_eu, perc_consolidation_cust_eu, on='ItemNumber', how='left')

In [ ]:
perc_consolidation_cust_apa = df_wide_data_items_in_order_apa.groupby('ItemNumber')['ConsolidationCustomer'].mean().reset_index()
perc_consolidation_cust_apa.columns = ['ItemNumber', 'ConsolidationCustomer']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_apa = pd.merge(aggregated_df_new_apa, perc_consolidation_cust_apa, on='ItemNumber', how='left')

In [ ]:
perc_consolidation_cust_usa = df_wide_data_items_in_order_usa.groupby('ItemNumber')['ConsolidationCustomer'].mean().reset_index()
perc_consolidation_cust_usa.columns = ['ItemNumber', 'ConsolidationCustomer']

# Merge the mean order quantity into emm_data_df
aggregated_df_new_usa = pd.merge(aggregated_df_new_usa, perc_consolidation_cust_usa, on='ItemNumber', how='left')

In [ ]:
perc_consolidation_cust = df_wide_data_items_in_order.groupby('ItemNumber')['ConsolidationCustomer'].mean().reset_index()
perc_consolidation_cust.columns = ['ItemNumber', 'ConsolidationCustomer']

# Merge the mean order quantity into emm_data_df
aggregated_df_new = pd.merge(aggregated_df_new, perc_consolidation_cust, on='ItemNumber', how='left')

In [ ]:
aggregated_df_new_usa.head()

In [ ]:
aggregated_df_new_eu.head()

In [ ]:
aggregated_df_new_apa.head()

# Add correlations and polynomial fit for numerical target

In [ ]:
aggregated_df_new_apa['ResampledCurrentGLP'] = aggregated_df_new_apa['CurrentGLP'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_eu['ResampledCurrentGLP'] = aggregated_df_new_eu['CurrentGLP'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_eu_apa['ResampledCurrentGLP'] = aggregated_df_new_eu_apa['CurrentGLP'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_usa['ResampledCurrentGLP'] = aggregated_df_new_usa['CurrentGLP'].apply(
    lambda arr: resample_array(np.array(arr), 4))

In [ ]:
aggregated_df_new_apa['ResampledCurrentDailyQty'] = aggregated_df_new_apa['CurrentDailyQty'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_eu['ResampledCurrentDailyQty'] = aggregated_df_new_eu['CurrentDailyQty'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_eu_apa['ResampledCurrentDailyQty'] = aggregated_df_new_eu_apa['CurrentDailyQty'].apply(
    lambda arr: resample_array(np.array(arr), 4))
aggregated_df_new_usa['ResampledCurrentDailyQty'] = aggregated_df_new_usa['CurrentDailyQty'].apply(
    lambda arr: resample_array(np.array(arr), 4))

In [ ]:
def compute_correlation(glp, qty):
    try:
        glp = np.array(glp)
        qty = np.array(qty)
        if len(glp) < 2 or len(qty) < 2:
            return np.nan
        if np.std(glp) == 0 or np.std(qty) == 0:
            return np.nan
        return np.corrcoef(glp, qty)[0, 1]
    except:
        return np.nan


def compute_poly_fit(glp, qty):
    try:
        if len(glp) < 2 or len(qty) < 2:
            return [np.nan, np.nan]
        coeffs = np.polyfit(glp, qty, deg=1)  # [slope, intercept]
        return coeffs.tolist()
    except:
        return [np.nan, np.nan]
def compute_poly_fit_deg(glp, qty, deg):
    try:
        if len(glp) <= deg or len(qty) <= deg:
            return [np.nan] * (deg + 1)
        coeffs = np.polyfit(glp, qty, deg=deg)
        return coeffs.tolist()
    except:
        return [np.nan] * (deg + 1)



In [ ]:

# Apply functions to each row
aggregated_df_new_apa["CorrGLPQty"] = aggregated_df_new_apa.apply(
    lambda row: compute_correlation(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)

aggregated_df_new_apa["PolyGLPQty"] = aggregated_df_new_apa.apply(
    lambda row: compute_poly_fit(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)
# Apply degree 2 polynomial fit
aggregated_df_new_apa["PolyDeg2GLPQty"] = aggregated_df_new_apa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=2
    ), axis=1
)

# Apply degree 3 polynomial fit
aggregated_df_new_apa["PolyDeg3GLPQty"] = aggregated_df_new_apa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=3
    ), axis=1
)

In [ ]:

# Apply functions to each row
aggregated_df_new_eu["CorrGLPQty"] = aggregated_df_new_eu.apply(
    lambda row: compute_correlation(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)

aggregated_df_new_eu["PolyGLPQty"] = aggregated_df_new_eu.apply(
    lambda row: compute_poly_fit(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)
# Apply degree 2 polynomial fit
aggregated_df_new_eu["PolyDeg2GLPQty"] = aggregated_df_new_eu.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=2
    ), axis=1
)

# Apply degree 3 polynomial fit
aggregated_df_new_eu["PolyDeg3GLPQty"] = aggregated_df_new_eu.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=3
    ), axis=1
)

In [ ]:

# Apply functions to each row
aggregated_df_new_eu_apa["CorrGLPQty"] = aggregated_df_new_eu_apa.apply(
    lambda row: compute_correlation(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)

aggregated_df_new_eu_apa["PolyGLPQty"] = aggregated_df_new_eu_apa.apply(
    lambda row: compute_poly_fit(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)
# Apply degree 2 polynomial fit
aggregated_df_new_eu_apa["PolyDeg2GLPQty"] = aggregated_df_new_eu_apa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=2
    ), axis=1
)

# Apply degree 3 polynomial fit
aggregated_df_new_eu_apa["PolyDeg3GLPQty"] = aggregated_df_new_eu_apa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=3
    ), axis=1
)

In [ ]:

# Apply functions to each row
aggregated_df_new_usa["CorrGLPQty"] = aggregated_df_new_usa.apply(
    lambda row: compute_correlation(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)

aggregated_df_new_usa["PolyGLPQty"] = aggregated_df_new_usa.apply(
    lambda row: compute_poly_fit(np.array(row["ResampledCurrentGLP"]), np.array(row["ResampledCurrentDailyQty"])),
    axis=1
)
# Apply degree 2 polynomial fit
aggregated_df_new_usa["PolyDeg2GLPQty"] = aggregated_df_new_usa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=2
    ), axis=1
)

# Apply degree 3 polynomial fit
aggregated_df_new_usa["PolyDeg3GLPQty"] = aggregated_df_new_usa.apply(
    lambda row: compute_poly_fit_deg(
        np.array(row["ResampledCurrentGLP"]), 
        np.array(row["ResampledCurrentDailyQty"]), 
        deg=3
    ), axis=1
)

In [ ]:
aggregated_df_new_usa.head()

In [ ]:
aggregated_df_new.head()

# Save final files

In [ ]:

aggregated_df_new.to_csv('AggregatedDataALL.csv', index=False)

aggregated_df_new_usa.to_csv('AggregatedDataUSA.csv', index=False)
aggregated_df_new_eu_apa.to_csv('AggregatedDataEUandAPA.csv', index=False)
aggregated_df_new.to_pickle('AggregatedDataALL.pkl')
aggregated_df_new_usa.to_pickle('AggregatedDataUSA.pkl')
aggregated_df_new_eu_apa.to_pickle('AggregatedDataEUandAPA.pkl')

In [ ]:
aggregated_df_new_eu.to_csv('AggregatedDataEU.csv', index=False)
aggregated_df_new_eu.to_pickle('AggregatedDataEU.pkl')

In [ ]:
aggregated_df_new_apa.to_csv('AggregatedDataAPA.csv', index=False)
aggregated_df_new_apa.to_pickle('AggregatedDataAPA.pkl')